# Pulmonary Chest X-Ray Abnormalities

## Dataset Kaggle
***
- [Pulmonary Chest X-Ray Abnormalities](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities)


## Contexto do problema
***
> A tuberculose é uma doença que afeta muitas pessoas nos países em desenvolvimento. Embora o tratamento seja possível, ela requer um diagnóstico preciso primeiro. Nos projetos de países, há em muitos casos máquinas de raio-X disponíveis (por meio de projetos de baixo custo e doações), mas muitas vezes falta a experiência radiológica para avaliar com precisão as imagens. Um algoritmo que pudesse realizar essa tarefa de forma rápida e barata poderia melhorar drasticamente a capacidade de diagnosticar e, em última análise, tratar a doença.  
***
> Em países mais desenvolvidos, a radiografia de raio-X é frequentemente usada para rastrear recém-chegados e determinar a elegibilidade para uma autorização de trabalho. A tarefa de examinar manualmente as imagens é demorada e um algoritmo pode aumentar a eficiência, melhorar o desempenho e, por fim, reduzir o custo dessa triagem.  
***
> Este conjunto de dados contém mais de 500 exames de raios-x com rótulos clínicos coletados por radiologistas.  

### Dataset 1: Montgomery County X-ray Set
***
As imagens de raios-X neste conjunto de dados foram adquiridas do programa de controle da tuberculose do Departamento de Saúde e Serviços Humanos do Condado de Montgomery, MD, EUA. Esse conjunto de dados contém 138 radiografias póstero-anterior, das quais 80 são radiografias normais e 58 são anormais com manifestações de tuberculose. Todas as imagens são desidentificadas e disponíveis no formato DICOM (Digital Imaging and Communications in Medicine). O conjunto cobre uma ampla gama de anormalidades, incluindo efusões e padrões miliares. O conjunto de dados inclui leituras de radiologia disponíveis como um arquivo de texto (preservando a identidade dos pacientes).

### Dataset 2: China Set - The Shenzhen set - Chest X-ray Database
***
O banco de dados de imagem digital padrão para tuberculose foi criado pela Biblioteca Nacional de Medicina, Maryland, EUA, em colaboração com o Hospital Popular de Shenzhen No.3, Faculdade de Medicina de Guangdong, Shenzhen, China. As radiografias de tórax são de clínicas ambulatoriais e foram capturadas como parte da rotina diária usando os sistemas Philips DR Digital Diagnose. O conjunto de dados contém 336 casos com manifestação de tuberculose e 326 casos normais.

### Referências
- Jaeger S, Candemir S, Antani S, Wáng YX, Lu PX, Thoma G. **Two public chest X-ray datasets for computer-aided screening of pulmonary diseases**. Quant Imaging Med Surg. 2014;4(6):475-477. doi:10.3978/j.issn.2223-4292.2014.11.20
- Jaeger S, Karargyris A, Candemir S, Folio L, Siegelman J, Callaghan F, Xue Z, Palaniappan K, Singh RK, Antani S, Thoma G, Wang YX, Lu PX, McDonald CJ. **Automatic tuberculosis screening using chest radiographs**. IEEE Trans Med Imaging. 2014 Feb;33(2):233-45. doi: 10.1109/TMI.2013.2284099. PMID: 24108713
- Candemir S, Jaeger S, Palaniappan K, Musco JP, Singh RK, Xue Z, Karargyris A, Antani S, Thoma G, McDonald CJ. **Lung segmentation in chest radiographs using anatomical atlases with nonrigid registration**. IEEE Trans Med Imaging. 2014 Feb;33(2):577-90. doi: 10.1109/TMI.2013.2290491. PMID: 24239990

## Importação dos pacotes

In [122]:
import glob
import re
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

import warnings
warnings.filterwarnings("ignore")

## Indexando todas as imagens disponíveis no dataset
***
- `glob`: encontra nomes de arquivos usando as regras usadas pelo shell Unix.

In [123]:
# coletando o caminho dos arquivos dos dados do hospital montgomery
filelist_montgomery = glob.glob('../datasets/archive/Montgomery/MontgomerySet/CXR_png/*.png')
# coletando o caminho dos arquivos dos dados do hospital shenzen
filelist_shenzen = glob.glob('../datasets/archive/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/*.png')
# unindo o caminho dos arquivos dos hospitais montgomery e shenzen
filelist = filelist_montgomery + filelist_shenzen

In [124]:
# quantidade de imagens disponíveis no dataset
print('quantidade de imagens:', str(len(filelist)))

quantidade de imagens: 800


## Extraindo os rótulos das imagens
***
- `regex`: módulo de expressão regular para encontrar cadeias de caracteres.
- `tqdm`: barra de carregamento gráfica para estruturas de repetição.

#### Rótulos
- `[0]` caso normal;
- `[1]` caso anormal;

In [125]:
def extract_label(file_list):
    
    # inicializando uma lista vazia
    labels = []
    
    # iterando na lista de arquivos
    for file in tqdm(file_list):
        # detectando as classes presentes no nome da imagem
        current_label = re.findall('[0-9]{4}_(.+?).png', file)
        # adicionando a lista de rótulos as classes correspondentes a cada uma das imagens
        labels.append(current_label[0])
        
    return labels

In [126]:
# extraindo os rótulos
labels = extract_label(filelist)

100%|██████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 6108.15it/s]


In [127]:
# visualizando a quantidade de rótulos
print('quantidade de rótulos:', str(len(labels)))

quantidade de rótulos: 800


## Criando um dataframe
***
- `pandas`: ferramenta de análise e manipulação de dados.

In [128]:
# criando um dataframe com os caminhos das imagens
full_data = pd.DataFrame(filelist, columns = ['filepath'])
# adicionando os rótulos em cada imagem
full_data['target'] = labels

In [129]:
dict_type = {'target': 'float32'}
full_data = full_data.astype(dict_type)

## Dados [Treinamento - Validação - Teste]
***
- `scikit-learn:` biblioteca de aprendizado de máquina de código aberto.

In [130]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [131]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [132]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['filepath']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['filepath']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['filepath']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 512
quantidade de rótulos de treinamento: 512
quantidade de imagens de teste: 160
quantidade de rótulos de teste: 160
quantidade de imagens de validação: 128
quantidade de rótulos de validação: 128


In [133]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True,
                                     rotation_range = 20, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'raw',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'filepath',
                                                  y_col = 'target',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'raw',
                                                  target_size = (256, 256))

Found 512 validated image filenames.
Found 128 validated image filenames.
Found 160 validated image filenames.


## Definição do modelo

In [134]:
# definindo o local do arquivo em que os pesos devem ser salvos 
filepath = "transferlearning_weights.hdf5" 
# definindo um callback de checkpoint para salvar os pesos durante o treinamento  
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [135]:
# definindo um callback de redução da taxa de aprendizado caso a rede entre em um platô  
lr_reduce = ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, min_delta = 1e-5, patience = 5, verbose = 1)

In [136]:
# definindo uma lista de callbacks
callbacks = [checkpoint, lr_reduce] 

In [137]:
# transferência de aprendizado
conv_base = VGG19(weights = 'imagenet', include_top = False, input_shape = (256, 256, 3))

In [138]:
# visualizando o a arquitetura VGG19 utilizada 
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [139]:
# definindo inicialmente toda rede como treinável 
conv_base.trainable = True
# criando uma flag 
set_trainable = False

# iterando entre as camadas da rede convolucional VGG19 
for layer in conv_base.layers:
    # caso estiver na última camada... 
    if layer.name == 'block5_conv1':
        # configure-a como treinável 
        set_trainable = True
    
    # caso a flag for verdadeira (na última camada)...
    if set_trainable:
        # mantenha a camada treinável 
        layer.trainable = True
    # caso a flag seja falsa (demais camadas)... 
    else:
        # mantenha as características aprendidas pelas camadas (não treináveis) 
        layer.trainable = False

In [140]:
# observando a arquitetura da rede após o refinamento
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [141]:
# definindo um modelo sequencial de rede neural 
model = models.Sequential()
# adicionando toda a arquitetura da VGG19 após o refinamento 
model.add(conv_base)
# adicionando uma camada de pooling (método do maior píxel) 
model.add(layers.MaxPooling2D())
# aplicando uma camada de normalização para otimizar a rede 
model.add(layers.BatchNormalization())
# aplicando uma camada de achatamento para conectar as camadas de convolução a uma rede neural densa 
model.add(layers.Flatten())
# definindo uma camada densa com 128 neurônios e uma função de ativação relu
model.add(layers.Dense(units = 128, activation = tf.nn.relu))
# aplicando uma camada de normalização na rede neural densa (zera 20% dos neurônios da camada anterior)
model.add(layers.Dropout(rate = 0.2))
# aplicando uma camada de saída com 15 unidades e uma função de ativação softmax 
model.add(layers.Dense(units = 2, activation = tf.nn.softmax)) 

In [142]:
# visualizando a nova arquitetura definida baseada em transferência de aprendizado
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 512)         2048      
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                

In [143]:
# compilando o modelo
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])

## Treinamento do modelo

In [ ]:
# treinando a rede 
history = model.fit_generator(train_generator,
                              steps_per_epoch = 512 // 32, 
                              validation_data = valid_generator,
                              validation_steps = 128 // 32,
                              callbacks = callbacks, epochs = 50, verbose = 1)

Epoch 1/50
16/16 [==============================] - 468s 28s/step - loss: 1.5112 - acc: 0.4831 - val_loss: 7.0906 - val_acc: 0.5078

Epoch 00001: val_acc improved from -inf to 0.50781, saving model to transferlearning_weights.hdf5
Epoch 2/50
16/16 [==============================] - 442s 27s/step - loss: 0.6969 - acc: 0.5194 - val_loss: 3.0523 - val_acc: 0.5078

Epoch 00002: val_acc did not improve from 0.50781
Epoch 3/50
16/16 [==============================] - 416s 26s/step - loss: 0.6932 - acc: 0.4954 - val_loss: 1.2784 - val_acc: 0.5078

Epoch 00003: val_acc did not improve from 0.50781
Epoch 4/50
 3/16 [====>.........................] - ETA: 6:11 - loss: 0.6932 - acc: 0.4583